In [66]:
import numpy as np
import pandas as pd
import os

def decimal2binary(dec_number):
    input_binary = bin(dec_number).replace("0b", "")
    #TM_binary = str('110') + input_binary + str('110')
    return input_binary
def expansion(n):
	n=decimal2binary(n)
	n=[int(x) for x in (n)]
	n2=[]
	n2.append(0)
	while len(n)!=0:
		if n[0]==0 & (len(n) >= 1):
			n2.append(0)
			del n[0]
		elif (n[0]==1) & (len(n) >= 1):
			n2.append(1)
			n2.append(0)
			del n[0]
		#print(n)
	n2.append(1)
	n2.append(1)
	n2.append(0)
	return n2

In [145]:
a=pd.read_csv("procesado.csv", header=None)
a.columns=(["Si","Si+1","Read","Write","Move"])
a

,Si,Si+1,Read,Write,Move
0,0,0,0,R,0.0
1,0,1,1,L,10000000.0
2,1,0,0,R,1.0
3,1,1,1,R,1001011.0
4,10,0,0,R,10.0
...,...,...,...,...,...
397,11000110,1,1,L,1100111.0
398,11000111,0,0,R,NaN
399,11000111,1,1,R,11001.0
400,11001000,0,0,R,100.0


In [ ]:
a.iloc[:]["Move"]=="H" #Verificación de que la MT ha parado

In [ ]:
a["Move"].where(a["Move"]=="H").index

In [ ]:
#este código que está por aquí 
idx = (a.index[a['Read']==1] & a.index[a["Move"]=="H"])
a.loc[idx]["Move"]

In [70]:
idx=(a.index[a['Read']==0] & a.index[a["Si"]==0])
idx

C:\Users\emman\AppData\Local\Temp\ipykernel_23240\521132656.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  idx=(a.index[a['Read']==0] & a.index[a["Si"]==0])


Int64Index([0], dtype='int64')

In [ ]:
estado=a.loc[idx]["Si+1"].item()
estado

In [72]:
def decode(instrucciones, n):
    cinta=expansion(n)
    estado=0
    posicion=0
    #grafo={}
    while True:
        print(cinta)
        print()
        if estado=="H": 
            break
        state=estado
        if (cinta[posicion]==0):
            idx=(instrucciones.index[instrucciones['Read']==0] & instrucciones.index[instrucciones["Si"]==estado])
            estado=instrucciones.loc[idx]["Si+1"].item()
            cinta[posicion]=instrucciones.loc[idx]["Write"].item()
            if instrucciones.loc[idx]["Move"].item()=="H":
                posicion=posicion
                break
            elif instrucciones.loc[idx]["Move"].item()=="R":
                posicion=posicion+1
            elif instrucciones.loc[idx]["Move"].item()=="L":
                posicion=posicion-1  
        elif (cinta[posicion]==1):
            idx=(instrucciones.index[instrucciones['Read']==1] & instrucciones.index[instrucciones["Si"]==estado])
            estado=instrucciones.loc[idx]["Si+1"].item()
            cinta[posicion]=instrucciones.loc[idx]["Write"].item()
            if instrucciones.loc[idx]["Move"].item()=="H":
                posicion=posicion
                break
            elif instrucciones.loc[idx]["Move"].item()=="R":
                posicion=posicion+1
            elif instrucciones.loc[idx]["Move"].item()=="L":
                posicion=posicion-1
        #print(cinta)
    return cinta

In [139]:
def run_utm(
        state = None,
        blank = None,
        rules = [],
        tape = [],
        halt = None,
        pos = 0):
    st = state
    if not tape: tape = [blank]
    if pos < 0: pos += len(tape)
    if pos >= len(tape) or pos < 0: raise "bad init position"
    rules = dict(((s0, v0), (v1, dr, s1)) for (s0, v0, v1, dr, s1) in rules)
    
    while True:
        print(st, '\t', end=" ")
        for i, v in enumerate(tape):
            if i == pos: print("[%s]" % (v,), end=" ")
            else: print(v, end=" ")
        print()
 
        if st == halt: break
        if (st, tape[pos]) not in rules: break
 
        (v1, dr, s1) = rules[(st, tape[pos])]
        tape[pos] = v1
        if dr == 'L':
            if pos > 0: pos -= 1
            else: tape.insert(0, blank)
        if dr == 'R':
            pos += 1
            if pos >= len(tape): tape.append(blank) 
        st = s1
    #return rules

In [140]:
print("\nbusy beaver\n")
run_utm(
    halt = 'H',
	state = '0',
	blank = '-',
    tape=expansion(34324),
	rules = map(tuple,a.round(0).values.tolist())
    )


busy beaver

0 	 [0] 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 


In [141]:
 rules=map(tuple,["A 1 1 right A".split(),
		"A 2 3 right B".split(),
		"A 0 0 left  E".split(),
		"B 1 1 right B".split(),
		"B 2 2 right B".split(),
		"B 0 0 left  C".split(),
		"C 1 2 left  D".split(),
		"C 2 2 left  C".split(),
		"C 3 2 left  E".split(),
		"D 1 1 left  D".split(),
		"D 2 2 left  D".split(),
		"D 3 1 right A".split(),
		"E 1 1 left  E".split(),
		"E 0 0 right STOP".split()])
rules = dict(((s0, v0), (v1, dr, s1)) for (s0, v0, v1, dr, s1) in rules)
rules

{('A', '1'): ('1', 'right', 'A'),
 ('A', '2'): ('3', 'right', 'B'),
 ('A', '0'): ('0', 'left', 'E'),
 ('B', '1'): ('1', 'right', 'B'),
 ('B', '2'): ('2', 'right', 'B'),
 ('B', '0'): ('0', 'left', 'C'),
 ('C', '1'): ('2', 'left', 'D'),
 ('C', '2'): ('2', 'left', 'C'),
 ('C', '3'): ('2', 'left', 'E'),
 ('D', '1'): ('1', 'left', 'D'),
 ('D', '2'): ('2', 'left', 'D'),
 ('D', '3'): ('1', 'right', 'A'),
 ('E', '1'): ('1', 'left', 'E'),
 ('E', '0'): ('0', 'right', 'STOP')}

In [142]:
rules=map(tuple,a.round(0).values.tolist())
rules = dict(((s0, v0), (v1, dr, s1)) for (s0, v0, v1, dr, s1) in rules)
rules

{(0, 0): (0, 'R', 0.0),
 (0, 1): (1, 'L', 10000000.0),
 (1, 0): (0, 'R', 1.0),
 (1, 1): (1, 'R', 1001011.0),
 (10, 0): (0, 'R', 10.0),
 (10, 1): (0, 'R', 10011000.0),
 (11, 0): (0, 'R', 11.0),
 (11, 1): (1, 'R', 1001110.0),
 (100, 0): (0, 'R', 100.0),
 (100, 1): (1, 'R', 1001101.0),
 (101, 0): (0, 'L', 101.0),
 (101, 1): (1, 'L', 1011010.0),
 (110, 0): (0, 'L', 110.0),
 (110, 1): (1, 'L', 1110101.0),
 (111, 0): (0, 'R', 111.0),
 (111, 1): (1, 'R', 10011001.0),
 (1000, 0): (0, 'R', 1000.0),
 (1000, 1): (1, 'R', 10100001.0),
 (1001, 0): (0, 'L', 1001.0),
 (1001, 1): (1, 'L', 10110001.0),
 (1010, 0): (0, 'L', 1010.0),
 (1010, 1): (1, 'L', 10111000.0),
 (1011, 0): (0, 'R', 1011.0),
 (1011, 1): (0, 'L', 1000100.0),
 (1100, 0): (0, 'L', 1100.0),
 (1100, 1): (0, 'L', 11000101.0),
 (1101, 0): (1, 'R', 10011.0),
 (1101, 1): (1, 'R', 1101.0),
 (1110, 0): (0, 'R', 1000.0),
 (1110, 1): (1, 'R', 1110.0),
 (1111, 0): (0, 'R', 101010.0),
 (1111, 1): (0, 'R', 1000000.0),
 (10000, 0): (0, 'L', 10000.0)

In [119]:
rules = map(tuple,
        ["a 0 1 right b".split(),
         "a 1 1 left  c".split(),
         "b 0 1 left  a".split(),
         "b 1 1 right b".split(),
         "c 0 1 left  b".split(),
         "c 1 1 stay  halt".split()])

In [102]:
"a 0 1 right b".split()

['a', '0', '1', 'right', 'b']